In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [74]:
#The purpose of this book is to compare in one plot the results of varying parameters.

FM2xy_camera_1000ms_40x = np.genfromtxt('./FM2xy_camera_1000ms_40x.csv',delimiter =",")
FM2xy_camera_1000ms_60x = np.genfromtxt('./FM2xy_camera_1000ms_60x.csv',delimiter =",")
FM2xy_camera_625ms_40x = np.genfromtxt('./FM2xy_camera_625ms_40x.csv',delimiter =",")
FM2xy_camera_625ms_60x = np.genfromtxt('./FM2xy_camera_625ms_60x.csv',delimiter =",")

FM2xy_confocal_1000ms_40x = np.genfromtxt('./FM2xy_confocal_1000ms_40x.csv',delimiter =",")
FM2xy_confocal_1000ms_60x = np.genfromtxt('./FM2xy_confocal_1000ms_60x.csv',delimiter =",")
FM2xy_confocal_625ms_40x = np.genfromtxt('./FM2xy_confocal_625ms_40x.csv',delimiter =",")
FM2xy_confocal_625ms_60x = np.genfromtxt('./FM2xy_confocal_625ms_60x.csv',delimiter =",")[0:-1]

In [85]:
geoM2xy_camera_1000ms_60x = np.genfromtxt('./geoM2xy_camera_1000ms_60x.csv',delimiter =",")
arM2xy_camera_1000ms_60x = np.genfromtxt('./arM2xy_camera_1000ms_60x.csv',delimiter =",")

In [26]:
f1 = 1.62
f2 = 1.02

time1 = np.zeros(FM2xy_camera_625ms_40x.shape[0])
time2 = np.zeros(FM2xy_camera_1000ms_40x.shape[0])

for num in range(0, FM2xy_camera_625ms_40x.shape[0]):
    time1[num] = num/f1

for num in range(0, FM2xy_camera_1000ms_40x.shape[0]):
    time2[num] = num/f2

In [24]:
FM2xy_confocal_1000ms_40x

array([        nan,  0.00793567,  0.00681826,  0.00725611,  0.00802544,
        0.00794948,  0.00762507,  0.00877513,  0.00693434,  0.0084937 ,
        0.00841248,  0.00884374,  0.00930203,  0.00819191,  0.00901503,
        0.00912503,  0.01021313,  0.01019277,  0.01044036,  0.01006949,
        0.01030254,  0.01033532,  0.01160115,  0.01355012,  0.01198642,
        0.01131481,  0.01217285,  0.01124666,  0.01309879,  0.01134813,
        0.01327542,  0.01201142,  0.01106507,  0.01164769,  0.01168304,
        0.01372017,  0.01252873,  0.01336069,  0.01416753,  0.01549862,
        0.01491109,  0.01530654,  0.01469845,  0.01484872,  0.01374063,
        0.01572874])

In [28]:
time2

array([  0.        ,   0.98039216,   1.96078431,   2.94117647,
         3.92156863,   4.90196078,   5.88235294,   6.8627451 ,
         7.84313725,   8.82352941,   9.80392157,  10.78431373,
        11.76470588,  12.74509804,  13.7254902 ,  14.70588235,
        15.68627451,  16.66666667,  17.64705882,  18.62745098,
        19.60784314,  20.58823529,  21.56862745,  22.54901961,
        23.52941176,  24.50980392,  25.49019608,  26.47058824,
        27.45098039,  28.43137255,  29.41176471,  30.39215686,
        31.37254902,  32.35294118,  33.33333333,  34.31372549,
        35.29411765,  36.2745098 ,  37.25490196,  38.23529412,
        39.21568627,  40.19607843,  41.17647059,  42.15686275,
        43.1372549 ,  44.11764706])

In [46]:
#Figure parameters

filename = 'FBF_comparison_40x.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [49]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
ax.plot(time1, FM2xy_camera_625ms_40x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 40x')
ax.plot(time2, FM2xy_confocal_1000ms_40x, 'go', ms=mi, label='Confocal 1000ms 40x')
ax.plot(time1, FM2xy_confocal_625ms_40x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 40x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [35]:
#Figure parameters

filename = 'FBF_comparison_60x.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

'MSD ($\\mu$m$^2$)'

In [59]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2, FM2xy_camera_1000ms_60x, 'bo', ms=mi, label='Camera 1000ms 60x')
ax.plot(time1, FM2xy_camera_625ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
ax.plot(time2, FM2xy_confocal_1000ms_60x, 'go', ms=mi, label='Confocal 1000ms 60x')
ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [60]:
#Figure parameters

filename = 'FBF_comparison_625ms.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [63]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time1, FM2xy_camera_625ms_40x, 'bo', ms=mi, label='Camera 625ms 40x')
ax.plot(time1, FM2xy_camera_625ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
ax.plot(time1, FM2xy_confocal_625ms_40x, 'go', ms=mi, label='Confocal 625ms 40x')
ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [64]:
#Figure parameters

filename = 'FBF_comparison_1000ms.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [65]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
ax.plot(time2, FM2xy_camera_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 1000ms 60x')
ax.plot(time2, FM2xy_confocal_1000ms_40x, 'go', ms=mi, label='Confocal 1000ms 40x')
ax.plot(time2, FM2xy_confocal_1000ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 1000ms 60x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [66]:
#Figure parameters

filename = 'FBF_comparison_camera.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [67]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
ax.plot(time2, FM2xy_camera_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 1000ms 60x')
ax.plot(time1, FM2xy_camera_625ms_40x, 'go', ms=mi, label='Camera 625ms 40x')
ax.plot(time1, FM2xy_camera_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [68]:
#Figure parameters

filename = 'FBF_comparison_confocal.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [69]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2, FM2xy_confocal_1000ms_40x, 'bo', ms=mi, label='Confocal 1000ms 40x')
ax.plot(time2, FM2xy_confocal_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Confocal 1000ms 60x')
ax.plot(time1, FM2xy_confocal_625ms_40x, 'go', ms=mi, label='Confocal 625ms 40x')
ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [91]:
#Figure parameters

filename = 'FBF_geo_ar_comparison.png'

limit1=20.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.0051 #y range
tick2 = round(limit2, 3)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=3 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [97]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2[0:19], geoM2xy_camera_1000ms_60x[0:-1], 'b--', linewidth=wi, dashes=[d1,d2], label='Geometric')
ax.plot(time2[0:19], arM2xy_camera_1000ms_60x[0:-1], 'g--', linewidth=wi, dashes=[d1,d2], label='Arithmetic')
ax.plot(time2, FM2xy_camera_1000ms_60x, 'r--', linewidth=wi, dashes=[d1,d2], label='Frame-by-frame')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [84]:
geoM2xy_camera_1000ms_60x.shape

(46,)